In [1]:
import os
import time
from langchain.chains import MapReduceDocumentsChain, LLMChain, ReduceDocumentsChain, StuffDocumentsChain
from langchain.llms import CTransformers
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
os.cpu_count()

16

In [3]:
# Load LLM
config = {'max_new_tokens': 4096, 'temperature': 0.7, 'context_length': 4096}
llm = CTransformers(model="TheBloke/Mistral-7B-Instruct-v0.2-GGUF",
                    model_file="mistral-7b-instruct-v0.2.Q4_K_M.gguf",
                    config=config,
                    threads=os.cpu_count())

/home/rol3ert99/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 1 files: 100%|██████████| 1/1 [04:29<00:00, 269.32s/it]


In [4]:
# Reduce template and chain
predict_sentiment_template = """<s>[INST] As a market stock expert assess sentiment(positive, negative, neutral) related to Apple Company for following text.
Only evaluate based on the provided text: {headline}
write short one word answer.
Answer:  [/INST] </s>"""

predict_sentiment_prompt = PromptTemplate.from_template(predict_sentiment_template)
predict_sentiment_chain = LLMChain(llm=llm, prompt=predict_sentiment_prompt)

In [5]:
from langchain.docstore.document import Document

headline = """Apple reportedly abandons its electric car project as EV makers struggle"""

docs =  [Document(page_content=headline, metadata={"source": "local"})]

# split_docs = text_splitter.split_documents(docs)

print(docs)

[Document(page_content='Apple reportedly abandons its electric car project as EV makers struggle', metadata={'source': 'local'})]


In [6]:
# Run the chain
start_time = time.time()
result = predict_sentiment_chain.__call__(docs, return_only_outputs=True)
time_taken = time.time() - start_time
print(result, time_taken)

/home/rol3ert99/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'text': ' Negative.'} 7.605067729949951


In [9]:
import pandas as pd

df_cnbc = pd.read_csv('cnbc_scraper/cnbc_headlines.csv', header = None, names = ['headline', 'date'])

df_cnbc.head(10)

,headline,date
0,"Huawei’s profit doubled in 2023 as smartphone,...",3/29/2024
1,"Cramer on Apple: Own it, don’t trade it",3/29/2024
2,This semiconductor ETF is headed for its best ...,3/28/2024
3,Apple’s bad quarter and what history says will...,3/28/2024
4,Xiaomi releases electric car $4K cheaper than ...,3/28/2024
5,Pro Talks: Katie Stockton on the one stock wit...,3/28/2024
6,Here are Thursday’s biggest analyst calls: Nvi...,3/28/2024
7,TikTok makes $2.1 million TV ad buy as Senate ...,3/28/2024
8,Here’s a rapid-fire update on all 33 portfolio...,3/27/2024
9,Morgan Stanley’s Erik Woodring presents his bu...,3/27/2024


In [11]:
headlines = df_cnbc['headline'].values
headlines

array(['Huawei’s profit doubled in 2023 as smartphone, autos business picked up',
       'Cramer on Apple: Own it, don’t trade it',
       'This semiconductor ETF is headed for its best quarterly gain in a year. Here are ...',
       ...,
       'Jim Cramer: My worldview for the first half of 2023 and the stocks that will ...',
       'Stocks close lower on first trading day of 2023, hurt by slumping Apple, Tesla',
       'More social media regulation is coming in 2023, members of Congress say'],
      dtype=object)

In [12]:
def predict_sentiment(headline):
  docs =  [Document(page_content=headline, metadata={"source": "local"})]
  start_time = time.time()
  result = predict_sentiment_chain.__call__(docs, return_only_outputs=True)
  time_taken = time.time() - start_time
  print(result, time_taken)
  return result

In [13]:
sentiments = []
for headline in headlines:
    sentiments.append(predict_sentiment(headline))

{'text': ' Neutral (The text does not directly mention Apple or its sentiment towards the company.)'} 8.399237394332886
{'text': ' Positive (based on Cramer\'s recommendation to "own it").'} 7.145156383514404
{'text': ' Neutral (The text does not directly mention Apple or its sentiment towards the company.)'} 7.96535587310791
{'text': ' Negative'} 3.3908956050872803
{'text': ' neutral'} 3.870361566543579
{'text': ' Neutral (The text does not express clear positive or negative sentiment towards Apple.)'} 7.396656274795532
{'text': ' Neutral (The text does not provide any specific positive, negative, or sentiment-indicating information about Apple.)'} 10.911091327667236
{'text': " Neutral. The text does not express an opinion about Apple or its sentiment towards the company. It only reports on TikTok making a TV ad buy and the Senate reviewing a potential ban on the app, with no mention of Apple's involvement or stance on the matter."} 19.2371563911438
{'text': ' Neutral (The text does n

KeyboardInterrupt: 